# Do all data-generating calculations

In [ ]:
include("stuff.jl")

In [ ]:
data_cube_dir

In [ ]:
vars

In [ ]:
plotMAP(getCubeData(Cube(data_cube_dir)))

In [ ]:
plotTS(getCubeData(Cube(data_cube_dir)))

In [ ]:
showVarInfo(getCubeData(Cube(data_cube_dir)))

In [ ]:
cube_base = getCubeData(Cube(data_cube_dir), variable = vars, time = (Date("2001-01-01"), Date("2011-12-31")))

In [ ]:
# the fluxcom derived data sets show some artefacts in the antarctic region which we remove here

rmprocs(workers())
addprocs(20)

@loadOrGenerate2 cube_base_clean => joinpath(cubes_base_dir, "cube_base_clean") begin
    cube_base_clean = mapCube(
        cube_base,
        indims = InDims("Lon", "Lat"),
        outdims = OutDims("Lon", "Lat"),
        max_cache = 1e10
    ) do xout, xin
        xout .= xin
        xout[:, 601:720, :] .= missing
    end
end

rmprocs(workers())

In [ ]:
plotTS(cube_base)

In [ ]:
plotTS(cube_base)

In [ ]:
plotMAP(cube_base)

In [ ]:
plotMAP(cube_base)

In [ ]:
showVarInfo(cube_base)

# z-transform and PCA

In [ ]:
cube_base_clean

In [ ]:
if !isfile(joinpath(cubes_base_dir, "z_trans_pca.jld2"))
    # 3h30m on a single core Float64!
    # 3h30m on a single core Double64!
    #cube_table = @CubeTable value = cube_base axes = (variable, lat) fastest = variable
    #cov_matrix = fittable(cube_table, WeightedCovMatrix{Double64}(), :value, weight = (i -> abs(cosd(i.lat)))).o
    
    # 9m on a single core Double64
    # 30 sec on 40 cores Double64!!
    rmprocs(workers())
    addprocs(40)
    @everywhere begin
        using ESDL, WeightedOnlineStats, DoubleFloats
        cube_base_clean = loadCube(joinpath($cubes_base_dir, "cube_base_clean"))
        getCubeData(Cube($data_cube_dir), variable = $vars, time = (Date("2001-01-01"), Date("2011-12-31")))
        weights = getAxis(LatAxis, cube_base_clean).values |> 
            collect |> 
            x -> repeat(x, inner = size(cube_base_clean)[1]) |> 
            x -> cosd.(x)
        function fit_cov_fun(T, d, w)
            (i) -> begin
                d[:, :, i, :] |> 
                x -> reshape(x, prod(size(x)[1:2]), size(x)[end]) |>
                x -> fit!(WeightedCovMatrix(T), x, w)
            end
        end
        fit_cov = fit_cov_fun(Double64, cube_base_clean, weights)
    end
     
    cov_matrices = @showprogress pmap(fit_cov, 1:size(cube_base_clean)[3])
    
    for c in cov_matrices
        @show size(c)
    end
    
    rmprocs(workers())
    cov_matrix = reduce(merge!, deepcopy(cov_matrices))
    
    cube_z_trans, cube_pca = pca(convert(WeightedCovMatrix{Float64}, cov_matrix))
    
    cube_pca.proj[:] = fix_pca_directions(cube_pca.proj, getAxis(VariableAxis, cube_base_clean).values)
    
    save(joinpath(cubes_base_dir, "z_trans_pca.jld2"), "z_trans", cube_z_trans, "pca", cube_pca)
else
    cube_z_trans = load(joinpath(cubes_base_dir, "z_trans_pca.jld2"), "z_trans")
    cube_pca = load(joinpath(cubes_base_dir, "z_trans_pca.jld2"), "pca")
end


# Apply PCA

In [ ]:
rmprocs(workers())
addprocs(40);

@everywhere using StatsBase, WeightedOnlineStats, MultivariateStats, LinearAlgebra
@everywhere begin
    
    StatsBase.transform(t::StatsBase.AbstractDataTransform, x) = StatsBase.transform!(x, t, x)
    StatsBase.transform(t::StatsBase.AbstractDataTransform, x) = StatsBase.transform!(similar(x), t, x)
    function StatsBase.transform!(y::AbstractVecOrMat, t::ZScoreTransform, x::AbstractVecOrMat)
        d = t.len
        size(x,1) == size(y,1) == d || throw(DimensionMismatch("Inconsistent dimensions."))
        n = size(y,2)
        size(x,2) == n || throw(DimensionMismatch("Inconsistent dimensions."))

        m = t.mean
        s = t.scale

        if isempty(m)
            if isempty(s)
                if x !== y
                    copyto!(y, x)
                end
            else
                broadcast!(/, y, x, s)
            end
        else
            if isempty(s)
                broadcast!(-, y, x, m)
            else
                broadcast!((x, m, s) -> (x - m) / s, y, x, m, s)
            end
        end
        return y
    end

    MultivariateStats.transform(M::PCA, x::AbstractVecOrMat) = 
        mul!(similar(x), transpose(M.proj), centralize(x, M.mean))

end # @everywhere


@loadOrGenerate2 cube_pca_online => joinpath(cubes_base_dir, "cube_pca_online") begin
    cube_pca_online = mapCube(
            cube_base_clean, cube_z_trans, cube_pca, 
            indims = InDims("Variable", "Lon"), 
            outdims = OutDims(pca_axis, "Lon")
        ) do xout, xin, z_trans, cube_pca
        xxin = xin[:, :]
        idx = vec(.!mapreduce(ismissing, |, xxin, dims = 1))
        xxin2 = xxin[:, idx]
        xout[:] .= missing
        if length(xxin2) > 0
            xout[:, idx] = transform((z_trans, cube_pca), xxin2)
        end            
    end
end

rmprocs(workers())

# PCA method analysis

In [ ]:
cumulative_explained_variance = cumsum(cube_pca.prinvars ./ cube_pca.tprinvar)
expl_variance = cube_pca.prinvars ./ cube_pca.tprinvar

In [ ]:
addprocs(40);

@loadOrGenerate2 pca_msc_cube => joinpath(cubes_base_dir, "pca_msc_cube") begin
    pca_msc_cube = getMSC(cube_pca_online)
end

@loadOrGenerate2 pca_no_msc_cube => joinpath(cubes_base_dir, "pca_no_msc_cube") begin
    pca_no_msc_cube = removeMSC(cube_pca_online)
end
rmprocs(workers())

In [ ]:
rmprocs(workers())
addprocs(12)

@loadOrGenerate2 cube_extrema_msc => joinpath(cubes_base_dir, "cube_extrema_msc") begin
    cube_extrema_msc = mapCube(
        pca_msc_cube,
        indims = InDims("MSC", "Lon", "Lat"),
        outdims = OutDims(CategoricalAxis(:Extrema, ["min", "max"])),
        max_cache = 1e9
    ) do xout, xin
         xout[:] .= extrema(skipmissing(xin))
    end
end

@loadOrGenerate2 cube_extrema_no_msc => joinpath(cubes_base_dir, "cube_extrema_no_msc") begin
    cube_extrema_no_msc = mapCube(
        pca_no_msc_cube,
        indims = InDims("Time", "Lon", "Lat"),
        outdims = OutDims(CategoricalAxis(:Extrema, ["min", "max"])),
        max_cache = 5e9
    ) do xout, xin
        xout[:] .= extrema(skipmissing(xin))
    end
end

rmprocs(workers())

In [ ]:
n_seasons = 4
comp = 1
doy_idxs = (length(getAxis(MSCAxis, pca_msc_cube)) ÷ n_seasons) .* (0:n_seasons - 1) .+ 1 |> collect
ex_msc = [
    cube_extrema_msc[:, i] |> 
    x -> abs.(x) |> 
    x -> max(x...) |> 
    x -> (-x, x) 
    for i in 1:size(cube_extrema_msc, 2)
]
ex_no_msc = [
    cube_extrema_no_msc[:, i] |> 
    x -> abs.(x) |>
    x -> max(x...) |> 
    x -> (-x, x) 
    for i in 1:size(cube_extrema_no_msc, 2)
]

In [ ]:
rmprocs(workers())
addprocs(24)

@everywhere begin
    import ESDL
    ESDL.DAT.checkskip(::ESDL.DAT.NoFilter, x::AbstractArray) = false
end

@loadOrGenerate2 cube_time_space_mask => joinpath(cubes_base_dir, "cube_time_space_mask") begin
    cube_time_space_mask = mapCube(
        cube_pca_online,
        indims = InDims("PcaAxis", "Lon", filter = ESDL.DAT.NoFilter()),
        outdims = OutDims("Lon", outtype = Float32)
    ) do xout, xin
        xout[:] .= vec(.!any(ismissing, xin, dims = 1))
    end
end

rmprocs(workers())

In [ ]:
rmprocs(workers())
addprocs(40)

@everywhere begin
    import ESDL
    ESDL.DAT.checkskip(::ESDL.DAT.NoFilter, x::AbstractArray) = false
end

@loadOrGenerate2 cube_msc_time_space_mask => joinpath(cubes_base_dir, "cube_msc_time_space_mask") begin
    cube_msc_time_space_mask = mapCube(
        pca_msc_cube,
        indims = InDims("MSC", "PcaAxis", filter = ESDL.DAT.NoFilter()),
        outdims = OutDims("MSC", outtype = Float32)
    ) do xout, xin
        xout[:] .= vec(.!any(ismissing, xin, dims = 2))
    end
end

rmprocs(workers())

# Variances of PC anomalies

In [ ]:
addprocs(40)
@everywhere using Statistics
pca_axis = CategoricalAxis(:PcaAxis, ["PCA_$i" for i in 1:length(vars)])
@loadOrGenerate2 pca_no_msc_std_cube => joinpath(cubes_base_dir, "pca_no_msc_std_cube") begin
    pca_no_msc_std_cube = mapCube(
        (xout, xin) -> xout[:] = std(xin, dims = 1), 
        pca_no_msc_cube,
        indims = InDims("Time", "PcaAxis"),
        outdims = OutDims(pca_axis)
    )
end
rmprocs(workers())

# Dryness index
the dryness index is calculated as $\frac P {PET}$

In [ ]:
p_pet_cube = getCubeData(
    Cube(data_cube_dir), 
    variable = ["precipitation", "potential_evaporation"], 
    time = (Date("2001-01-01"), Date("2011-12-31"))
)

In [ ]:
rmprocs(procs())
addprocs(20)

@everywhere begin
    function aridity1(xout, xin)
        p_pet_sum = 0.0
        n = 0
        for i in 1:size(xin, 2)
            p = xin[1, i]
            pet = xin[2, i]
            p_pet = p / pet
            if !(ismissing(p_pet) || isnan(p_pet) || isinf(p_pet) || p_pet <= 0)
                n += 1
                p_pet_sum += p_pet
            end
        end
        if n > 50
            xout[1] = p_pet_sum / n
        else
            xout[1] = missing
        end
    end

    function aridity2(xout, xin)
        p_sum = 0.0
        p_n = 0
        pet_sum = 0.0
        pet_n = 0
        for i in 1:size(xin, 2)
            p = xin[1, i]
            pet = xin[2, i]
            if !(ismissing(p) || isnan(p))
                p_sum += p
                p_n += 1
            end
            if !(ismissing(pet) || isnan(pet) || pet <= 0)
                pet_sum += pet
                pet_n += 1
            end
        end

        if p_n > 50 && pet_n > 50 && pet_sum > 0
            p_mean = p_sum / p_n
            pet_mean = pet_sum / pet_n
            ai = p_mean / pet_mean
            xout[1] = ai
        else
            xout[1] = missing
        end
    end
end

@loadOrGenerate2 dryness_cube1 => joinpath(cubes_base_dir, "dryness_cube1") begin
    dryness_cube1 = mapCube(
        aridity1,
        p_pet_cube,
        indims = InDims("Variable", "Time"),
        outdims = OutDims()
    ) 
end

@loadOrGenerate2 dryness_cube2 => joinpath(cubes_base_dir, "dryness_cube2") begin
    dryness_cube2 = mapCube(
        aridity2,
        p_pet_cube,
        indims = InDims("Variable", "Time"),
        outdims = OutDims()
    ) 
end

rmprocs(procs())

# Sen's Slope

In [ ]:
rmprocs(procs())
addprocs(40)

@everywhere begin
    using Distributions

    count_duplicates(x) = filter!(xx -> xx > 1, map(v -> count(y -> y === v, skipmissing(x)), unique(x)))

    function theil_sen(y::AbstractVector{Tm}, alpha::T = T(0.95)) where Tm <: Union{Missing, T} where T
        MAX_MISSING = 3
        
        ny = length(y)
        ns = ((ny ^ 2) - ny) ÷ 2

        ss = Array{T}(undef, ns)
        S::Int64 = 0
        k = 1
        @inbounds for i in 1:(ny - 1)
            for j in (i + 1):ny
                delta_y = y[j] - y[i]
                if ismissing(delta_y)
                    ss[k] = NaN
                else
                    tmp = delta_y / (j - i)
                    S += Int64(sign(delta_y))
                    ss[k] = tmp
                end
                k += 1
            end
        end
        filter!(!isnan, ss)
        sort!(ss)

        nt = length(ss)
        if nt < ns - MAX_MISSING
            return missing, (missing, missing), missing
        end

        med_slope = isodd(nt) ? ss[nt ÷ 2 + 1] : ((ss[nt ÷ 2] + ss[nt ÷ 2 + 1]) / 2)
        # this one is really not interesting if there are no x values
        #med_int = median(y) - med_slope * (T(1 / 2) + T(ny / 2))

        if alpha > 0.5
            alpha = 1.0 - alpha
        end

        z = T(quantile(Normal(), alpha / 2))
        nyreps = count_duplicates(y)

        # if there are x values, duplicates have to be substracted the same way as they are for the y vals!
        sigsq = T(1/18) * (ny * (ny - 1) * (2 * ny + 5) -
                        mapreduce(k -> k * (k - 1) * (2 * k + 5), +, nyreps, init = zero(eltype(nyreps))))
        sigma = sqrt(sigsq)

        # find interval
        ru = min(round(Int64, (nt - z * sigma) / 2) + 1, nt)
        rl = max(round(Int64, (nt + z * sigma) / 2),     1)

        delta = (ss[rl], ss[ru])
        
        # calculate p-value
        Z = (S - sign(S)) / sigma
        p = T(2 * (1 - cdf(Normal(), abs(Z))))
        

        return med_slope, 
            #med_int, 
            delta,
            p
    end

end

@loadOrGenerate2 pca_trends => joinpath(cubes_base_dir, "pca_trends") pca_trends_pvals => joinpath(cubes_base_dir, "pca_trends_pvals") begin
    pca_trends, pca_trends_pvals = mapCube(
        cube_pca_online, 
        indims = InDims("Time"), 
        outdims = (OutDims(), OutDims())
    ) do xout_pca_trends, xout_pca_trends_pvals, xin
        
        slope, (sl, su), p = theil_sen(xin)
        
        xout_pca_trends[1]       = slope
        xout_pca_trends_pvals[1] = p
        
        return nothing
    end
end

#=
@loadOrGenerate2 pca_trends_2 => joinpath(cubes_base_dir, "pca_trends_2") begin
    pca_trends_2 = mapCube(
        cube_pca_online, 
        indims = InDims("Time"), 
        outdims = OutDims()
    ) do xout, xin
        slope, delta = theil_sen(xin)
        xout[1] = slope
    end
end
=#

rmprocs(procs())


In [ ]:
plotMAP(pca_trends)

In [ ]:
plotMAP(pca_trends_pvals)

In [ ]:
function cummin(x::Array{Tm}) where Tm <: Union{Missing, T} where T
    m = typemax(T)
    xout = similar(x)
    @inbounds for i in eachindex(x, xout)
        if ismissing(x[i])
            xout[i:end] .= missing
            return xout
        elseif x[i] < m
            xout[i] = x[i]
            m = x[i]
        else
            xout[i] = m
        end
    end
    return xout
end

function bh_adj(p)
    # copied from R::stats::p.adjust
    # order behaves the same way as sortperm with respect to missing values/NAs, except that R will always put missing values last, even if the sorting order is decreasing! 
    # and therefore this should be enough to deal with missing values
    nomissp = filter(!ismissing, p)
    pout = similar(p)
    
    lp = length(nomissp)
    i = lp:-1:1
    o = sortperm(nomissp, rev = true)
    ro = sortperm(o)
    nomissp_adj = min.(1, cummin(lp ./ i .* nomissp[o]))[ro]

    j = 1
    @inbounds for i in eachindex(p)
        if ismissing(p[i])
            pout[i] = missing
        else
            pout[i] = nomissp_adj[j]
            j += 1
        end
    end
    
    @assert j == length(nomissp_adj) + 1
    
    return pout
end

In [ ]:
@loadOrGenerate2 pca_trends_pvals_adj => joinpath(cubes_base_dir, "pca_trends_pvals_adj") begin
    pca_trends_pvals_adj = pca_trends_pvals[:, :, :] |> 
        vec |> 
        collect |> 
        bh_adj |> 
        x -> reshape(x, size(pca_trends_pvals)) |>
        collect |>
        x -> CubeMem(pca_trends_pvals.axes, x, Dict{String, String}())
end

In [ ]:
plotMAP(pca_trends_pvals_adj)

In [ ]:
pca_trends_pvals_adj[:, :, :] |> vec |> collect |> skipmissing |> extrema

In [ ]:
function test_ts() 
    n = 1000
    t = Array{Float64}(undef, n)
    nsig = 0
    nnosig = 0
    for i in 1:n
        d = rand(1000) .+ 0.0001 .* collect(1:1000)
        t[i] = @elapsed s, (sl, su), p = theil_sen(d)
        if ((sl <= 0 <= su) && (p > 0.05))
            nsig += 1
        elseif ((sl > 0) || (su < 0)) && (p < 0.05)
            nnosig += 1
        else
            println("error")
            @show nsig nnosig mean(t)
            return d
        end
    end
    println("ok")
    @show nsig nnosig mean(t)
    return [1.0]
end        
d = test_ts()

In [ ]:
#### this is outdated!!!! TODO: do the same as with the pc slopes and adjust the p values

pt_cube = getCubeData(Cube(data_cube_dir), variable = ["air_temperature_2m", "precipitation"], time = (Date("2001-01-01"), Date("2011-12-31")))

rmprocs(procs())
addprocs(40)

@everywhere begin
    using Distributions

    count_duplicates(x) = filter!(xx -> xx > 1, map(v -> count(y -> y === v, skipmissing(x)), unique(x)))

    function theil_sen(y::AbstractVector{Union{Missing, T}}, alpha::T = T(0.95)) where T
        MAX_MISSING = 3
        
        ny = length(y)
        ns = ((ny ^ 2) - ny) ÷ 2

        ss = Array{T}(undef, ns)
        k = 1
        @inbounds for i in 1:(ny - 1)
            for j in (i + 1):ny
                tmp = (y[i] - y[j]) / (i - j)
                ss[k] = !ismissing(tmp) ? tmp : T(NaN)
                k += 1
            end
        end
        filter!(!isnan, ss)
        sort!(ss)

        nt = length(ss)
        if nt < ns - MAX_MISSING
            return missing, (missing, missing)
        end

        med_slope = isodd(nt) ? ss[nt ÷ 2 + 1] : ((ss[nt ÷ 2] + ss[nt ÷ 2 + 1]) / 2)
        # this one is really not interesting if there are no x values
        #med_int = median(y) - med_slope * (T(1 / 2) + T(ny / 2))

        if alpha > 0.5
            alpha = 1.0 - alpha
        end

        z = T(quantile(Normal(), alpha / 2))
        nyreps = count_duplicates(y)

        # if there are x values, duplicates have to be substracted the same way as they are for the y vals!
        sigsq = T(1/18) * (ny * (ny - 1) * (2 * ny + 5) -
                        mapreduce(k -> k * (k - 1) * (2 * k + 5), +, nyreps, init = zero(eltype(nyreps))))
        sigma = sqrt(sigsq)

        ru = min(round(Int64, (nt - z * sigma) / 2) + 1,     nt)
        rl = max(round(Int64, (nt + z * sigma) / 2), 1)

        delta = (ss[rl], ss[ru])

        return med_slope, 
            #med_int, 
            delta
    end

end

@loadOrGenerate2 pt_trends => joinpath(cubes_base_dir, "pt_trends") begin
    pt_trends = mapCube(
        pt_cube, 
        indims = InDims("Time"), 
        outdims = OutDims()
    ) do xout, xin
        slope, delta = theil_sen(xin)
        if ismissing(slope)
            xout[1] = missing
        elseif (slope + delta[1]) <= 0 <= (slope + delta[2])
            xout[1] = 0
        else
            xout[1] = slope
        end
    end
end

@loadOrGenerate2 pt_trends_2 => joinpath(cubes_base_dir, "pt_trends_2") begin
    pt_trends_2 = mapCube(
        pt_cube, 
        indims = InDims("Time"), 
        outdims = OutDims()
    ) do xout, xin
        slope, delta = theil_sen(xin)
        xout[1] = slope
    end
end


rmprocs(procs())

In [ ]:
rmprocs(procs())
addprocs(40)

@everywhere begin
    using Distributions

    count_duplicates(x) = filter!(xx -> xx > 1, map(v -> count(y -> y === v, skipmissing(x)), unique(x)))

    function theil_sen(
            y::AbstractVector{Union{Missing, T}}, 
            x::AbstractVector, 
            alpha::T = T(0.95)
        ) where T
        
        ny = length(y)
        ns = ((ny ^ 2) - ny) ÷ 2

        ss = Array{T}(undef, ns)
        k = 1
        @inbounds for i in 1:(ny - 1)
            for j in (i + 1):ny
                tmp = (y[i] - y[j]) / (x[i] - x[j])
                ss[k] = !ismissing(tmp) ? tmp : T(NaN)
                k += 1
            end
        end
        filter!(z -> !ismissing(z) && !isnan(z), ss)
        sort!(ss)

        nt = length(ss)
        if nt == 0
            return missing, (missing, missing)
        end

        med_slope = isodd(nt) ? ss[nt ÷ 2 + 1] : ((ss[nt ÷ 2] + ss[nt ÷ 2 + 1]) / 2)
        # this one is really not interesting if there are no x values
        #med_int = median(y) - med_slope * (T(1 / 2) + T(ny / 2))

        if alpha > 0.5
            alpha = 1.0 - alpha
        end

        z = T(quantile(Normal(), alpha / 2))
        nyreps = count_duplicates(y)

        # if there are x values, duplicates have to be substracted the same way as they are for the y vals!
        sigsq = T(1/18) * (ny * (ny - 1) * (2 * ny + 5) -
                  mapreduce(k -> k * (k - 1) * (2 * k + 5), +, nyreps, init = zero(eltype(nyreps))))
        sigma = sqrt(sigsq)

        ru = min(round(Int64, (nt - z * sigma) / 2) + 1, nt)
        rl = max(round(Int64, (nt + z * sigma) / 2),     1)

        delta = (ss[rl], ss[ru])

        return med_slope, 
            #med_int, 
            delta
    end
    
    ntimesteps = 4
    tvals = repeat(0.0f0:46:505, inner = ntimesteps)
    idxs =  vec( (0:(0 + ntimesteps - 1)) .+ (0:46:505)' )
    @assert length(tvals) == length(idxs)

end

@loadOrGenerate2 pca_trends_season => joinpath(cubes_base_dir, "pca_trends_season") begin
    pca_trends_season = mapCube(
        cube_pca_online, tvals,
        indims = InDims("Time"), 
        outdims = OutDims(MSCAxis(46))
    ) do xout, xin, tvals
        
        for i in 1:length(xout)

            idxs2 = mod1.(idxs .+ i, length(xin))
            slope, delta = theil_sen(xin[idxs2], tvals)
            
            if ismissing(slope)
                xout[i] = missing
            elseif slope + delta[1] < 0 < slope + delta[2]
                xout[i] = 0
            elseif isinf(slope)
                xout[i] = missing
            else
                xout[i] = slope
            end
        end
    end
end

rmprocs(procs())

# Breakpoint detection

In [ ]:
# setting breakpoints

rmprocs(procs())
addprocs(24)

@everywhere begin
    using RCall
end
@everywhere begin
    R"""suppressWarnings(library(strucchange, quietly = TRUE))"""
end

@everywhere missing_to_nan(x::Array{Union{Missing, T}}) where T = map(y -> y === missing ? T(NaN) : y, x)

@loadOrGenerate2 cube_break_points => joinpath(cubes_base_dir, "cube_break_points") begin
    cube_break_points = 
        mapCube(
            cube_pca_online, 
            indims = InDims("Time"), 
            outdims = OutDims()
        ) do xout, xin
            if any(ismissing, xin)
                xout[1] = missing
            else
                xin2 =  convert(Array{Float64}, xin)
                idx = Float32.(R"""
                    data <- $(xin2)
                    model <- efp(data ~ 1)
                    sig <- sctest(model)$p.value < 0.05
                    
                    if (sig) {
                        idx <- breakpoints(data ~ 1, breaks = 1)$breakpoints
                        if(is.na(idx)) {
                            0
                        } else {
                            idx
                        }
                    } else {
                        0
                    }
                """)
                if length(idx) == 1
                    xout[1] = idx[1]
                elseif length(idx) == 0
                    xout[1] = missing
                else
                    error("length idx is other than 0 or 1")
                end
            end
        end
end

rmprocs(procs())

In [ ]:
cube_break_points

In [ ]:
R"""

unique($(cube_break_points[1, :, :] |> vec |> collect |> missing_to_nan))

"""

In [ ]:
plotMAP(cube_break_points)

# Yearly amplitude

In [ ]:
rmprocs(workers())
addprocs(40)

year_axis = RangeAxis("Year", sort(unique(year.(cube_base.timeAxis.values))))

@loadOrGenerate2 y_amplitude_cube => joinpath(cubes_base_dir, "y_amplitude_cube") begin
    
    y_amplitude_cube = mapCube(
        cube_pca_online,
        indims = InDims(pca_axis, TimeAxis),
        outdims =  OutDims(pca_axis, year_axis)
    ) do xout, xin
        if any(ismissing, xin) || any(isnan, xin)
            xout[:] .= missing
        else

            for j in axes(xout, 1)
                for i in axes(xout, 2)
                    start_idx = 1 + 46 * (i - 1)
                    end_idx   = 46 + 46 * (i - 1)
                    ex = extrema(xin[j, start_idx:end_idx])
                    xout[j, i] = ex[2] - ex[1]
                end
            end
            
        end
    end
end

rmprocs(workers())

In [ ]:
rmprocs(procs())
addprocs(40)

@everywhere begin

    using Distributions

    count_duplicates(x) = filter!(xx -> xx > 1, map(v -> count(y -> y === v, skipmissing(x)), unique(x)))

    function theil_sen(y::AbstractVector{Tm}, alpha::T = T(0.95)) where Tm <: Union{Missing, T} where T
        MAX_MISSING = 3
        
        ny = length(y)
        ns = ((ny ^ 2) - ny) ÷ 2

        ss = Array{T}(undef, ns)
        S::Int64 = 0
        k = 1
        @inbounds for i in 1:(ny - 1)
            for j in (i + 1):ny
                delta_y = y[j] - y[i]
                if ismissing(delta_y)
                    ss[k] = NaN
                else
                    tmp = delta_y / (j - i)
                    S += Int64(sign(delta_y))
                    ss[k] = tmp
                end
                k += 1
            end
        end
        filter!(!isnan, ss)
        sort!(ss)

        nt = length(ss)
        if nt < ns - MAX_MISSING
            return missing, (missing, missing), missing
        end

        med_slope = isodd(nt) ? ss[nt ÷ 2 + 1] : ((ss[nt ÷ 2] + ss[nt ÷ 2 + 1]) / 2)
        # this one is really not interesting if there are no x values
        #med_int = median(y) - med_slope * (T(1 / 2) + T(ny / 2))

        if alpha > 0.5
            alpha = 1.0 - alpha
        end

        z = T(quantile(Normal(), alpha / 2))
        nyreps = count_duplicates(y)

        # if there are x values, duplicates have to be substracted the same way as they are for the y vals!
        sigsq = T(1/18) * (ny * (ny - 1) * (2 * ny + 5) -
                        mapreduce(k -> k * (k - 1) * (2 * k + 5), +, nyreps, init = zero(eltype(nyreps))))
        sigma = sqrt(sigsq)

        # find interval
        ru = min(round(Int64, (nt - z * sigma) / 2) + 1, nt)
        rl = max(round(Int64, (nt + z * sigma) / 2),     1)

        delta = (ss[rl], ss[ru])
        
        # calculate p-value
        Z = (S - sign(S)) / sigma
        p = T(2 * (1 - cdf(Normal(), abs(Z))))
        

        return med_slope, 
            #med_int, 
            delta,
            p
    end

    
    ntimesteps = 4
    tvals = repeat(0.0f0:46:505, inner = ntimesteps)
    idxs =  vec( (0:(0 + ntimesteps - 1)) .+ (0:46:505)' )
    @assert length(tvals) == length(idxs)

end


@loadOrGenerate2 pca_y_amplitude_trends => joinpath(cubes_base_dir, "pca_y_amplitude_trends") pca_y_amplitude_trends_pvals => joinpath(cubes_base_dir, "pca_y_amplitude_trends_pvals") begin
    pca_y_amplitude_trends, pca_y_amplitude_trends_pvals = mapCube(
        y_amplitude_cube,
        indims = InDims("Year"), 
        outdims = (OutDims(), OutDims())
    ) do xout_trends, xout_pvals, xin
        
        slope, (sl, su), p = theil_sen(xin)
        
        xout_trends[1] = slope
        xout_pvals[1] = p

        return nothing
    end
end

rmprocs(procs())

In [ ]:
pca_y_amplitude_trends_pvals

In [ ]:
#@loadOrGenerate2 pca_y_amplitude_trends_pvals_adj => joinpath(cubes_base_dir, "pca_y_amplitude_trends_pvals_adj") begin
    pca_y_amplitude_trends_pvals_adj = pca_y_amplitude_trends_pvals[:, :, :] |>  
        collect |>
        vec |>
        bh_adj |> 
        x -> reshape(x, size(pca_y_amplitude_trends_pvals)) |>
        collect |> 
        x -> CubeMem(pca_y_amplitude_trends_pvals.axes, x, Dict{String, String}())
#end

In [ ]:
plotMAP(pca_y_amplitude_trends_pvals_adj)

In [ ]:
plotMAP(pca_y_amplitude_trends_pvals_adj)

In [ ]:
plotMAP(pca_y_amplitude_trends)

In [ ]:
pca_y_amplitude_trends_pvals[:, :, :] |> vec |> skipmissing |> extrema

In [ ]:
pca_y_amplitude_trends_pvals_adj[:, :, :] |> vec |> skipmissing |> extrema

# Reconstruction Error

In [ ]:
rmprocs(procs())
addprocs(40)

@loadOrGenerate2 pca_recon_error_1 => joinpath(cubes_base_dir, "pca_recon_error_1") begin
    pca_recon_error_1 = mapCube(
        cube_base_clean,
        cube_pca.proj[:, 1:1] * cube_pca.proj[:, 1:1]', 
        cube_z_trans,
        indims = InDims("Variable"),
        outdims = OutDims("Variable")
    ) do xout, xin, reproj, ztransform

        if any(ismissing, xin)
            xout[:] .= missing
        else
            
            xin .= (xin .- ztransform.mean) ./ ztransform.scale
            xinrecon = reproj * xin
            xout[:] .= xin .- xinrecon

        end
        
    end
        
end

rmprocs(procs())

In [ ]:
rmprocs(procs())
addprocs(40)

@loadOrGenerate2 pca_recon_error_2 => joinpath(cubes_base_dir, "pca_recon_error_2") begin
    pca_recon_error_2 = mapCube(
        cube_base_clean,
        cube_pca.proj[:, 1:2] * cube_pca.proj[:, 1:2]', 
        cube_z_trans,
        indims = InDims("Variable"),
        outdims = OutDims("Variable")
    ) do xout, xin, reproj, ztransform

        if any(ismissing, xin)
            xout[:] .= missing
        else
            
            xin .= (xin .- ztransform.mean) ./ ztransform.scale
            xinrecon = reproj * xin
            xout[:] .= xin .- xinrecon

        end
        
    end
        
end

rmprocs(procs())

In [ ]:
rmprocs(procs())
addprocs(40)

@loadOrGenerate2 pca_recon_error_3 => joinpath(cubes_base_dir, "pca_recon_error_3") begin
    pca_recon_error_3 = mapCube(
        cube_base_clean,
        cube_pca.proj[:, 1:3] * cube_pca.proj[:, 1:3]', 
        cube_z_trans,
        indims = InDims("Variable"),
        outdims = OutDims("Variable")
    ) do xout, xin, reproj, ztransform

        if any(ismissing, xin)
            xout[:] .= missing
        else
            
            xin .= (xin .- ztransform.mean) ./ ztransform.scale
            xinrecon = reproj * xin
            xout[:] .= xin .- xinrecon

        end
        
    end
        
end

rmprocs(procs())

In [ ]:
rmprocs(procs())
addprocs(40)

@everywhere using Statistics

@loadOrGenerate2 pca_recon_error_1_sp => joinpath(cubes_base_dir, "pca_recon_error_1_sp") begin
    pca_recon_error_1_sp = mapCube(
        pca_recon_error_1,
        indims = InDims("Variable", "Time"),
        outdims = OutDims()
    ) do xout, xin

        if all(ismissing, xin)
            xout[1] = missing
        else
            
            xout[1] = mean(x -> x^2, (skipmissing(xin)))

        end
        
    end
        
end

rmprocs(procs())

In [ ]:
rmprocs(procs())
addprocs(40)

@everywhere using Statistics

@loadOrGenerate2 pca_recon_error_2_sp => joinpath(cubes_base_dir, "pca_recon_error_2_sp") begin
    pca_recon_error_2_sp = mapCube(
        pca_recon_error_2,
        indims = InDims("Variable", "Time"),
        outdims = OutDims()
    ) do xout, xin

        if all(ismissing, xin)
            xout[1] = missing
        else
            
            xout[1] = mean(x -> x^2, (skipmissing(xin)))

        end
        
    end
        
end

rmprocs(procs())

In [ ]:
rmprocs(procs())
addprocs(40)

@everywhere using Statistics

@loadOrGenerate2 pca_recon_error_3_sp => joinpath(cubes_base_dir, "pca_recon_error_3_sp") begin
    pca_recon_error_3_sp = mapCube(
        pca_recon_error_3,
        indims = InDims("Variable", "Time"),
        outdims = OutDims()
    ) do xout, xin

        if all(ismissing, xin)
            xout[1] = missing
        else
            
            xout[1] = mean(x -> x^2, (skipmissing(xin)))

        end
        
    end
        
end

rmprocs(procs())

In [ ]:
rmprocs(procs())
addprocs(40)

@everywhere using Statistics

@loadOrGenerate2 pca_recon_error_1_lat => joinpath(cubes_base_dir, "pca_recon_error_1_lat") begin
    pca_recon_error_1_lat = mapCube(
        pca_recon_error_1,
        indims = InDims("Variable", "Lon", "Time"),
        outdims = OutDims(),
        max_cache = 1e9
    ) do xout, xin

        if all(ismissing, xin)
            xout[1] = missing
        else
            
            xout[1] = mean(x -> x^2, (skipmissing(xin)))

        end
        
    end
        
end

rmprocs(procs())

In [ ]:
rmprocs(procs())
addprocs(40)

@everywhere using Statistics

@loadOrGenerate2 pca_recon_error_2_lat => joinpath(cubes_base_dir, "pca_recon_error_2_lat") begin
    pca_recon_error_2_lat = mapCube(
        pca_recon_error_2,
        indims = InDims("Variable", "Lon", "Time"),
        outdims = OutDims(),
        max_cache = 1e9
    ) do xout, xin

        if all(ismissing, xin)
            xout[1] = missing
        else
            
            xout[1] = mean(x -> x^2, (skipmissing(xin)))

        end
        
    end
        
end

rmprocs(procs())

In [ ]:
rmprocs(procs())
addprocs(40)

@everywhere using Statistics

@loadOrGenerate2 pca_recon_error_3_lat => joinpath(cubes_base_dir, "pca_recon_error_3_lat") begin
    pca_recon_error_3_lat = mapCube(
        pca_recon_error_3,
        indims = InDims("Variable", "Lon", "Time"),
        outdims = OutDims(),
        max_cache = 1e9
    ) do xout, xin

        if all(ismissing, xin)
            xout[1] = missing
        else
            
            xout[1] = mean(x -> x^2, (skipmissing(xin)))

        end
        
    end
        
end

rmprocs(procs())